In [37]:
from datetime import datetime
from geopy.geocoders import Nominatim
import pandas as pd
import numpy as np
import requests
import json
from tqdm import tqdm
from time import sleep
import os


#import sys 
#sys.path.append("../")   aquí no es necesario porque tengo el src a la misma altura que el jupyter

from src.funciones import info_vuelos, crearDF_formateado, info_hoteles, crearDF_formateado_hoteles



In [38]:
hotel_tene_1=pd.read_csv("datos/hotel_tene_1.csv")
hotel_tene_2=pd.read_csv("datos/hotel_tene_2.csv")
hotel_tene_3=pd.read_csv("datos/hotel_tene_3.csv")
hotel_tene_4=pd.read_csv("datos/hotel_tene_4.csv")


hotel_pmi_1=pd.read_csv("datos/hotel_pmi_1.csv")
hotel_pmi_2=pd.read_csv("datos/hotel_pmi_2.csv")
hotel_pmi_3=pd.read_csv("datos/hotel_pmi_3.csv")
hotel_pmi_4=pd.read_csv("datos/hotel_pmi_4.csv")
hotel_pmi_5=pd.read_csv("datos/hotel_pmi_5.csv")
hotel_pmi_6=pd.read_csv("datos/hotel_pmi_6.csv")
hotel_pmi_7=pd.read_csv("datos/hotel_pmi_7.csv")
hotel_pmi_8=pd.read_csv("datos/hotel_pmi_8.csv")


***EDA***

In [39]:
df_hoteles_pmi=pd.DataFrame()
df_hoteles_pmi= pd.concat([df_hoteles_pmi,hotel_pmi_1,hotel_pmi_2,hotel_pmi_3,hotel_pmi_4,hotel_pmi_5,hotel_pmi_6,hotel_pmi_7,hotel_pmi_8], ignore_index= True)

In [40]:
df_hoteles_tene=pd.DataFrame()
df_hoteles_tene= pd.concat([df_hoteles_tene,hotel_tene_1,hotel_tene_2,hotel_tene_3,hotel_tene_4], ignore_index= True)

In [41]:
df_hoteles_total= pd.DataFrame()
df_hoteles_total=pd.concat([df_hoteles_total,df_hoteles_pmi,df_hoteles_tene], ignore_index= True)

**Hoteles**

In [42]:
df_hoteles_pmi.describe().T


,count,mean,std,min,25%,50%,75%,max
reviews_hotel,216.0,322.356481,615.907421,0.0,2.00,39.0,336.75,3702.0
rating_hotel,216.0,3.537037,1.624956,0.0,3.00,4.0,4.50,5.0
precio_medio_por noche,155.0,230.874194,116.262796,30.0,150.25,219.5,300.25,631.5


In [43]:
df_hoteles_pmi.describe(include="O").T

,count,unique,top,freq
nombre_hotel,216,204,Metropolitan Juka Playa,3
ciudad,216,1,Palma de Mallorca,216
servicios,216,75,[],67
link,216,206,https://www.tripadvisor.com/Hotel_Review-g1874...,2


In [44]:
df_hoteles_pmi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216 entries, 0 to 215
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   nombre_hotel            216 non-null    object 
 1   reviews_hotel           216 non-null    int64  
 2   rating_hotel            216 non-null    float64
 3   precio_medio_por noche  155 non-null    float64
 4   ciudad                  216 non-null    object 
 5   servicios               216 non-null    object 
 6   link                    216 non-null    object 
dtypes: float64(2), int64(1), object(4)
memory usage: 11.9+ KB


**Hay datos nulos en el precio medio asi que habrá que gestionarlo en algun momento**  

se pierde un 33,94% de tenerife si se eliminan los nulos     
se pierde un 28,24% de pmi si se eliminan los nulos   
respecto al total, se pierde un 30% respecto al total

Me planteo rellenarlo con la media

In [45]:
df_hoteles_tene.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109 entries, 0 to 108
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   nombre_hotel            109 non-null    object 
 1   reviews_hotel           109 non-null    int64  
 2   rating_hotel            109 non-null    float64
 3   precio_medio_por noche  72 non-null     float64
 4   ciudad                  109 non-null    object 
 5   servicios               109 non-null    object 
 6   link                    109 non-null    object 
dtypes: float64(2), int64(1), object(4)
memory usage: 6.1+ KB


In [46]:
df_hoteles_tene.describe().T

,count,mean,std,min,25%,50%,75%,max
reviews_hotel,109.0,108.733945,374.916673,0.0,0.000,3.0,32.000,2829.0
rating_hotel,109.0,2.518349,1.995280,0.0,0.000,3.5,4.000,5.0
precio_medio_por noche,72.0,92.027778,36.722397,26.0,63.125,88.5,121.125,178.5


In [47]:
df_hoteles_tene.describe(include="O").T

,count,unique,top,freq
nombre_hotel,109,102,Montes de Anaga Hostel,2
ciudad,109,1,Santa Cruz de Tenerife,109
servicios,109,26,[],61
link,109,102,https://www.tripadvisor.com/Hotel_Review-g1874...,2


In [48]:
df_hoteles_total.head()

,nombre_hotel,reviews_hotel,rating_hotel,precio_medio_por noche,ciudad,servicios,link
0,Hotel Costa Azul,2081,4.5,177.0,Palma de Mallorca,"['Free Wifi', 'Pool', 'Indoor pool', 'Restaura...",https://www.tripadvisor.com/Hotel_Review-g1874...
1,Hotel Victoria Gran Meliá,2447,4.5,270.0,Palma de Mallorca,"['Free Wifi', 'Pool', 'Heated pool', 'Indoor p...",https://www.tripadvisor.com/Hotel_Review-g1874...
2,Nakar Hotel,2327,5.0,349.5,Palma de Mallorca,"['Free Wifi', 'Breakfast included', 'Pool', 'H...",https://www.tripadvisor.com/Hotel_Review-g1874...
3,INNSiDE Palma Bosque,1082,4.5,160.0,Palma de Mallorca,"['Free Wifi', 'Indoor pool', 'Outdoor pool', '...",https://www.tripadvisor.com/Hotel_Review-g1874...
4,BQ Belvedere Hotel,3702,4.0,154.0,Palma de Mallorca,"['Free parking', 'Pool', 'Outdoor pool', 'Rest...",https://www.tripadvisor.com/Hotel_Review-g1874...


In [49]:
df_hoteles_total.shape

(325, 7)

In [50]:
df_hoteles_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 325 entries, 0 to 324
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   nombre_hotel            325 non-null    object 
 1   reviews_hotel           325 non-null    int64  
 2   rating_hotel            325 non-null    float64
 3   precio_medio_por noche  227 non-null    float64
 4   ciudad                  325 non-null    object 
 5   servicios               325 non-null    object 
 6   link                    325 non-null    object 
dtypes: float64(2), int64(1), object(4)
memory usage: 17.9+ KB


Los describes no me sirven en el total porque lo que quiero es comparar. (Para visualizar haré group by)

In [51]:
df_hoteles_total.describe()

,reviews_hotel,rating_hotel,precio_medio_por noche
count,325.000000,325.000000,227.000000
mean,250.710769,3.195385,186.834802
std,555.681044,1.819682,117.592932
min,0.000000,0.000000,26.000000
25%,1.000000,2.000000,96.500000
50%,14.000000,4.000000,159.000000
75%,198.000000,4.500000,254.000000
max,3702.000000,5.000000,631.500000


**Vuelos**

In [52]:
vuelos_pmi=pd.read_csv("datos/vuelos_pmi.csv")
vuelos_tene=pd.read_csv("datos/vuelos_tene.csv")

In [53]:
vuelos_pmi.head()

,id_vuelo,id_origen,ciudad_origen,id_destino,ciudad_destino,salida,llegada,duracion(min),escalas,aerolinea,precio
0,13870-2411030835--31915-0-15434-2411031005,MAD,Madrid,PMI,Palma,2024-11-03 08:35:00,2024-11-03 10:05:00,90,0,Ryanair,53.92
1,15434-2411062220--31915-0-13870-2411062350,PMI,Palma,MAD,Madrid,2024-11-06 22:20:00,2024-11-06 23:50:00,90,0,Ryanair,53.92
2,13870-2411030705--32221-0-15434-2411030830,MAD,Madrid,PMI,Palma,2024-11-03 07:05:00,2024-11-03 08:30:00,85,0,Iberia Express,46.60
3,15434-2411062220--31915-0-13870-2411062350,PMI,Palma,MAD,Madrid,2024-11-06 22:20:00,2024-11-06 23:50:00,90,0,Ryanair,46.60
4,13870-2411031115--31915-0-15434-2411031245,MAD,Madrid,PMI,Palma,2024-11-03 11:15:00,2024-11-03 12:45:00,90,0,Ryanair,101.24


In [54]:
vuelos_pmi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id_vuelo        20 non-null     object 
 1   id_origen       20 non-null     object 
 2   ciudad_origen   20 non-null     object 
 3   id_destino      20 non-null     object 
 4   ciudad_destino  20 non-null     object 
 5   salida          20 non-null     object 
 6   llegada         20 non-null     object 
 7   duracion(min)   20 non-null     int64  
 8   escalas         20 non-null     int64  
 9   aerolinea       20 non-null     object 
 10  precio          20 non-null     float64
dtypes: float64(1), int64(2), object(8)
memory usage: 1.8+ KB


In [55]:
vuelos_pmi.describe().T

,count,mean,std,min,25%,50%,75%,max
duracion(min),20.0,86.750,2.446802,85.0,85.00,85.000,90.00,90.00
escalas,20.0,0.000,0.000000,0.0,0.00,0.000,0.00,0.00
precio,20.0,87.613,23.704194,46.6,68.08,102.555,103.87,112.09


In [56]:
vuelos_pmi.describe(include="O").T

,count,unique,top,freq
id_vuelo,20,12,13870-2411030705--32221-0-15434-2411030830,5
id_origen,20,2,MAD,10
ciudad_origen,20,2,Madrid,10
id_destino,20,2,PMI,10
ciudad_destino,20,2,Palma,10
salida,20,9,2024-11-03 07:05:00,7
llegada,20,9,2024-11-03 08:30:00,7
aerolinea,20,4,Iberia Express,8


In [66]:
vuelos_tene.head()

,id_vuelo,id_origen,ciudad_origen,id_destino,ciudad_destino,salida,llegada,duracion(min),escalas,aerolinea,precio
0,13870-2411030620--32221-0-16864-2411030815,MAD,Madrid,TFN,Tenerife,2024-11-03 06:20:00,2024-11-03 08:15:00,175,0,Iberia Express,152.20
1,16864-2411062125--32221-0-13870-2411070115,TFN,Tenerife,MAD,Madrid,2024-11-06 21:25:00,2024-11-07 01:15:00,170,0,Iberia Express,152.20
2,13870-2411030705--32680-0-16864-2411030855,MAD,Madrid,TFN,Tenerife,2024-11-03 07:05:00,2024-11-03 08:55:00,170,0,Air Europa,176.68
3,16864-2411061755--32680-0-13870-2411062140,TFN,Tenerife,MAD,Madrid,2024-11-06 17:55:00,2024-11-06 21:40:00,165,0,Air Europa,176.68
4,13870-2411030620--32221-0-16864-2411030815,MAD,Madrid,TFN,Tenerife,2024-11-03 06:20:00,2024-11-03 08:15:00,175,0,Iberia Express,176.93


In [58]:
vuelos_tene.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id_vuelo        20 non-null     object 
 1   id_origen       20 non-null     object 
 2   ciudad_origen   20 non-null     object 
 3   id_destino      20 non-null     object 
 4   ciudad_destino  20 non-null     object 
 5   salida          20 non-null     object 
 6   llegada         20 non-null     object 
 7   duracion(min)   20 non-null     int64  
 8   escalas         20 non-null     int64  
 9   aerolinea       20 non-null     object 
 10  precio          20 non-null     float64
dtypes: float64(1), int64(2), object(8)
memory usage: 1.8+ KB


In [59]:
vuelos_tene.describe().T

,count,mean,std,min,25%,50%,75%,max
duracion(min),20.0,170.250,4.722566,165.0,165.00,170.000,175.00,175.00
escalas,20.0,0.000,0.000000,0.0,0.00,0.000,0.00,0.00
precio,20.0,177.446,11.718194,152.2,172.58,178.015,188.13,188.43


In [60]:
vuelos_tene.describe(include="O").T

,count,unique,top,freq
id_vuelo,20,13,13870-2411030620--32221-0-16864-2411030815,5
id_origen,20,2,MAD,10
ciudad_origen,20,2,Madrid,10
id_destino,20,2,TFN,10
ciudad_destino,20,2,Tenerife,10
salida,20,8,2024-11-03 06:20:00,9
llegada,20,8,2024-11-03 08:15:00,9
aerolinea,20,3,Iberia Express,10


In [61]:
vuelos_tene.shape

(20, 11)

In [62]:
vuelos_pmi.shape

(20, 11)

**Duplicados**  
  
No me preocupan los duplicados porque al estar ofreciendo paquetes de ida y vuelta, puede haber casos en los que el paquete tenga el mismo vuelo de ida pero distintos vuelos de vuelta. 

Por eso decido dejarlos para que cuenten también como opcion.

In [68]:
vuelos_pmi.duplicated().sum()

2

In [67]:
vuelos_tene.duplicated().sum()

2